In [35]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
file = 'logins.json'
logins = pd.read_json(file)
logins.head()

,login_time
0,1970-01-01 20:13:18
1,1970-01-01 20:16:10
2,1970-01-01 20:16:37
3,1970-01-01 20:16:36
4,1970-01-01 20:26:21


In [3]:
logins['15_minute_intervals'] = [i*15 for i in range(logins.size)]
logins.head()

,login_time,15_minute_intervals
0,1970-01-01 20:13:18,0
1,1970-01-01 20:16:10,15
2,1970-01-01 20:16:37,30
3,1970-01-01 20:16:36,45
4,1970-01-01 20:26:21,60


In [4]:
logins['base_time'] = dt.datetime(logins['login_time'].dt.year[0],
                                  logins['login_time'].dt.month[0],
                                  logins['login_time'].dt.day[0],
                                  logins['login_time'].dt.hour[0],
                                  minute=0)
logins.head()

,login_time,15_minute_intervals,base_time
0,1970-01-01 20:13:18,0,1970-01-01 20:00:00
1,1970-01-01 20:16:10,15,1970-01-01 20:00:00
2,1970-01-01 20:16:37,30,1970-01-01 20:00:00
3,1970-01-01 20:16:36,45,1970-01-01 20:00:00
4,1970-01-01 20:26:21,60,1970-01-01 20:00:00


In [5]:
logins['interval_start'] = logins['base_time'] + pd.TimedeltaIndex(logins['15_minute_intervals'], unit='m')
logins.head()

,login_time,15_minute_intervals,base_time,interval_start
0,1970-01-01 20:13:18,0,1970-01-01 20:00:00,1970-01-01 20:00:00
1,1970-01-01 20:16:10,15,1970-01-01 20:00:00,1970-01-01 20:15:00
2,1970-01-01 20:16:37,30,1970-01-01 20:00:00,1970-01-01 20:30:00
3,1970-01-01 20:16:36,45,1970-01-01 20:00:00,1970-01-01 20:45:00
4,1970-01-01 20:26:21,60,1970-01-01 20:00:00,1970-01-01 21:00:00


In [9]:
logins['interval_end'] = logins['interval_start'] + pd.Timedelta(15, unit='m')
logins.head()

,login_time,15_minute_intervals,base_time,interval_start,interval_end
0,1970-01-01 20:13:18,0,1970-01-01 20:00:00,1970-01-01 20:00:00,1970-01-01 20:15:00
1,1970-01-01 20:16:10,15,1970-01-01 20:00:00,1970-01-01 20:15:00,1970-01-01 20:30:00
2,1970-01-01 20:16:37,30,1970-01-01 20:00:00,1970-01-01 20:30:00,1970-01-01 20:45:00
3,1970-01-01 20:16:36,45,1970-01-01 20:00:00,1970-01-01 20:45:00,1970-01-01 21:00:00
4,1970-01-01 20:26:21,60,1970-01-01 20:00:00,1970-01-01 21:00:00,1970-01-01 21:15:00


In [23]:
bins = pd.IntervalIndex.from_tuples(
    [
        (logins.iloc[i]['interval_start'], logins.iloc[i]['interval_end'])
        for i in range(logins.shape[0])
    ]
)
logins['binning'] = pd.cut(logins['login_time'], bins=bins)
logins.head()

,login_time,15_minute_intervals,base_time,interval_start,interval_end,binning
0,1970-01-01 20:13:18,0,1970-01-01 20:00:00,1970-01-01 20:00:00,1970-01-01 20:15:00,"(1970-01-01 20:00:00, 1970-01-01 20:15:00]"
1,1970-01-01 20:16:10,15,1970-01-01 20:00:00,1970-01-01 20:15:00,1970-01-01 20:30:00,"(1970-01-01 20:15:00, 1970-01-01 20:30:00]"
2,1970-01-01 20:16:37,30,1970-01-01 20:00:00,1970-01-01 20:30:00,1970-01-01 20:45:00,"(1970-01-01 20:15:00, 1970-01-01 20:30:00]"
3,1970-01-01 20:16:36,45,1970-01-01 20:00:00,1970-01-01 20:45:00,1970-01-01 21:00:00,"(1970-01-01 20:15:00, 1970-01-01 20:30:00]"
4,1970-01-01 20:26:21,60,1970-01-01 20:00:00,1970-01-01 21:00:00,1970-01-01 21:15:00,"(1970-01-01 20:15:00, 1970-01-01 20:30:00]"


In [33]:
binned_logins = logins[['login_time', 'binning']].groupby('binning').agg('count')
binned_logins.head()

,login_time
binning,
"(1970-01-01 20:00:00, 1970-01-01 20:15:00]",2
"(1970-01-01 20:15:00, 1970-01-01 20:30:00]",6
"(1970-01-01 20:30:00, 1970-01-01 20:45:00]",9
"(1970-01-01 20:45:00, 1970-01-01 21:00:00]",7
"(1970-01-01 21:00:00, 1970-01-01 21:15:00]",1


In [39]:
binned_logins.rename({'login_time':'count'}, axis=1, inplace=True)
binned_logins.head()

,count
binning,
"(1970-01-01 20:00:00, 1970-01-01 20:15:00]",2
"(1970-01-01 20:15:00, 1970-01-01 20:30:00]",6
"(1970-01-01 20:30:00, 1970-01-01 20:45:00]",9
"(1970-01-01 20:45:00, 1970-01-01 21:00:00]",7
"(1970-01-01 21:00:00, 1970-01-01 21:15:00]",1


In [43]:
# plot using a line plot. too many elements for a bar plot